<a href="https://colab.research.google.com/github/glnrzr/TezCalismasi/blob/master/example1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 133872 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.18-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [15]:
!mkdir -p drive
!google-drive-ocamlfuse drive

/bin/bash: line 0: cd: too many arguments


In [22]:
cd /content/drive/My Drive/Colab Notebooks/Advances in Fuzzy Systems

/content/drive/My Drive/Colab Notebooks/Advances in Fuzzy Systems


In [0]:
import os

In [60]:
miniData = []
filenames = ['A Collaborative Framework for Privacy Preserving Fuzzy Co-Clustering of Vertically Distributed Cooccurrence Matrices.txt', 'A Coupled Fixed Point Theorem in Fuzzy Metric Space Satisfying -Contractive Condition.txt', 'A Coupled Fixed Point Theorem in Fuzzy Metric Space Satisfying -Contractive Condition.txt']
for fname in filenames:
    with open(fname) as infile:
        for line in infile:
            try:
              _Y, _X = line.strip().split("\t")
              miniData.append([_X, _Y])
            except Exception as e:
              print (line)
              #print(e)
              pass
np.random.shuffle(miniData)
train_x = [x[0] for x in miniData]
train_y = np.asarray([x[1] for x in miniData])
miniData = None
#print train_x[0] , " " , train_y[0]
#print train_x[1] , " " , train_y[1]
num_of_classes = len(np.unique(train_y))

In many real world data analysis tasks, 

it is expected that we can get much more

 useful knowledge by utilizing multiple 

 databases stored in different organizations,

  such as cooperation groups, state organs,

   and allied countries. However,

    in many such organizations, 

    they often hesitate to publish 

    their databases because of privacy and 

    security issues although they believe

     the advantages of collaborative analysis.

      This paper proposes a novel collaborative 

      framework for utilizing vertically

       partitioned cooccurrence matrices 

       in fuzzy co-cluster structure estimation, 

        which cooccurrence information among 

        objects and items is separately stored

         in several sites. In order to utilize

          such distributed data sets without

           fear of information leaks,

            a privacy preserving procedure 

            is introduced to fuzzy clustering 

            for categorical multi

In [61]:
print(train_y)
print(miniData)

[]
None


In [57]:
i = 0
for val in train_y:
  if val == '3':
    i = i+1
print (i)

0


In [54]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(train_y)
encoded_Y = encoder.transform(train_y)
# convert integers to dummy variables (i.e. one hot encoded)
train_y = np_utils.to_categorical(encoded_Y)

ValueError: ignored

In [0]:
import json
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer

max_words = 3000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_x)
dictionary = tokenizer.word_index

with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)


def convert_text_to_index_array(text):
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]


allWordIndices = []
for text in train_x:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

allWordIndices = np.asarray(allWordIndices)
train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
#train_y = keras.utils.to_categorical(train_y, 4)

In [38]:
convert_text_to_index_array("komutan suriyede bin askerimiz var dogal")

KeyError: ignored

In [0]:
import h5py
with h5py.File('train_x.h5', 'w') as hf:
    hf.create_dataset("three-class-all",  data=train_x)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model = Sequential()
model.add(Dense(512, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(num_of_classes, activation='softmax'))

In [45]:
model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])

model.fit(train_x, train_y,
  batch_size=32,
  epochs=5,
  verbose=1,
  validation_split=0.1,
  shuffle=True)

RuntimeError: ignored